# Draw fully-resolved dependency graph

Draw dependency graph for recipes listed in `crum.yaml` file, where the radius of
each node is proportional to the number of incoming edges, i.e., dependencies.

In [ ]:
import os
import sys

# %matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
import networkx as nx
import path_helpers as ph

root = ph.path('.').realpath().parent.parent
if root not in sys.path:
    sys.path.insert(0, root)
import crum
import crum.recipes
import crum.git_tools as gt


# Load recipes.
crum_root = crum.find_crum_root()
recipe_objs_ = crum.recipes.load_recipe_objs(crum_root.joinpath('crum.yaml'), verbose=False)

# Determine package dependency build order.
dependency_graph = crum.recipes.dependency_graph(recipe_objs_,
                                                 all_dependencies=False)

# Prune non-explicit dependencies (i.e., dependencies not explicitly listed in the recipe).
to_prune = [edge_i for edge_i in dependency_graph.edges
            if edge_i[0] not in dependency_graph
            .nodes[edge_i[1]]['recipe'].text()]
dependency_graph.remove_edges_from(to_prune)

G_ = dependency_graph.copy()

build_order = []
positions = []

# Find each level of dependency packages until all packages have had
# dependencies met.
while G_.number_of_nodes():
    # Find nodes with no incoming edges, indicating all dependencies are
    # available at this step.
    root_nodes = [node_i for node_i in G_.nodes if not G_.in_degree(node_i)]
    # Use spring layout within each level of dependencies.
    positions += [nx.spring_layout(G_.subgraph(root_nodes))]
    # Add packages ready for build at this step.
    build_order += [root_nodes]
    # Remove built packages of this step from the graph.
    G_.remove_nodes_from(root_nodes)
build_order

# Set `x`-position to the depth of the dependency in the graph.
pos = {}
for i, (level_i, positions_i) in enumerate(zip(build_order, positions)):
    for j, package_j in enumerate(sorted(level_i, key=lambda x:
                                         dependency_graph.out_degree(x))):
        pos[package_j] = positions_i[package_j].copy()
        pos[package_j][0] = i
        
# Draw the dependency graph, save to PDF, and open.
fig, axis = plt.subplots(1, figsize=(len(build_order) * 5,
                                     2.5 * max(map(len, build_order))))
nx.draw_networkx(dependency_graph, pos=pos,
                 node_size=[(15 + 2 * (dependency_graph.in_degree(k) + 1)) ** 2
                            for k in dependency_graph.nodes])
output = ph.path('dependency_graph.pdf')
fig.savefig(output, bbox='tight')
output.launch()